In [1]:
import pandas as pd
from etl.data_loader import DataLoader

/var/folders/ct/fjh_5m0n2zj9r5lpl1nrd20m0000gn/T/ipykernel_32265/3591165556.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [7]:
results = DataLoader().get_latest_data()
df = pd.DataFrame(results)

# Define a mapping dictionary
mapping_dict = {'1': 'A/B', '2': 'A/C', '3': 'B/C'}

# Replace the numerical suffixes in column names
df.columns = df.columns.to_series().replace(mapping_dict, regex=True)

# Display the modified DataFrame
pd.set_option('display.max_rows', None)
columns_to_check = ['sub_pair_B/C', 'sub_pair_A/B', 'sub_pair_A/C', 'bid_price_B/C', 'bid_price_A/B', 'bid_price_A/C',
                     'ask_price_B/C', 'ask_price_A/B', 'ask_price_A/C']

df[columns_to_check] = df[columns_to_check].replace('0', pd.NA)
df = df.dropna(subset=columns_to_check)

price_columns = ['bid_price_B/C', 'bid_price_A/B', 'bid_price_A/C', 
                  'ask_price_B/C', 'ask_price_A/B', 'ask_price_A/C']

df[price_columns] = df[price_columns].astype(float)
df['price_A/C_2'] = df['ask_price_A/B'] * df['ask_price_B/C']
df['is_s<__1'] = df['ask_price_A/B'] * df['ask_price_B/C'] * (1 / df['ask_price_A/C'])

df['is_g>__1'] = df['bid_price_A/B'] * df['bid_price_B/C'] * (1 / df['bid_price_A/C'])
df['s<1_diff'] = 1 - df['is_s<__1']
df['g>1_diff'] = df['is_g>__1'] - 1
df['diff'] = df[['s<1_diff', 'g>1_diff']].max(axis=1)
df['chosen_diff'] = df[['s<1_diff', 'g>1_diff']].idxmax(axis=1)
# Sort by the maximum diff value
df.loc[(df['s<1_diff'] < 0) & (df['g>1_diff'] < 0), 'chosen_diff'] = '-'
sorted_df = df.sort_values(by='diff', ascending=False)
print(len(sorted_df))
display(sorted_df)

# ETH-BTC-USDT	ETHBTC	0.053420	0.053430	ETHUSDT	2300.14	2300.15	BTCUSDT	4.304101e+04	4.304102e+04	0.999796	0.999613

32


,parent_pair,sub_pair_A/B,bid_price_A/B,ask_price_A/B,sub_pair_A/C,bid_price_A/C,ask_price_A/C,sub_pair_B/C,bid_price_B/C,ask_price_B/C,price_A/C_2,is_s<__1,is_g>__1,s<1_diff,g>1_diff,diff,chosen_diff
33,ETH-USDT-BIDR,ETHUSDT,2301.030000,2301.040000,ETHBIDR,36015111.00,36233341.00,USDTBIDR,1.572100e+04,1.572500e+04,3.618385e+07,0.998634,1.004425,1.365786e-03,0.004425,4.425410e-03,g>1_diff
11,ETH-BTC-BIDR,ETHBTC,0.053610,0.053620,ETHBIDR,36015111.00,36233341.00,BTCBIDR,6.746890e+08,6.758298e+08,3.623799e+07,1.000128,1.004303,-1.284068e-04,0.004303,4.302794e-03,g>1_diff
93,ETH-EUR-AEUR,ETHEUR,2134.270000,2134.280000,ETHAEUR,2126.00,2134.54,EURAEUR,9.997000e-01,9.999000e-01,2.134067e+03,0.999778,1.003589,2.217939e-04,0.003589,3.588767e-03,g>1_diff
38,ETH-USDT-UAH,ETHUSDT,2301.030000,2301.040000,ETHUAH,102257.00,102748.00,USDTUAH,4.457000e+01,4.459000e+01,1.026034e+05,0.998592,1.002933,1.407584e-03,0.002933,2.932876e-03,g>1_diff
16,ETH-BTC-UAH,ETHBTC,0.053610,0.053620,ETHUAH,102257.00,102748.00,BTCUAH,1.912823e+06,1.915172e+06,1.026915e+05,0.999450,1.002831,5.496687e-04,0.002831,2.830525e-03,g>1_diff
21,ETH-BTC-AEUR,ETHBTC,0.053610,0.053620,ETHAEUR,2126.00,2134.54,BTCAEUR,3.972112e+04,3.981416e+04,2.134835e+03,1.000138,1.001622,-1.383245e-04,0.001622,1.622410e-03,g>1_diff
8,ETH-BTC-ZAR,ETHBTC,0.053610,0.053620,ETHZAR,44381.00,44466.00,BTCZAR,8.286120e+05,8.287140e+05,4.443564e+04,0.999317,1.000921,6.826636e-04,0.000921,9.213249e-04,g>1_diff
30,ETH-USDT-ZAR,ETHUSDT,2301.030000,2301.040000,ETHZAR,44381.00,44466.00,USDTZAR,1.930000e+01,1.932000e+01,4.445609e+04,0.999777,1.000651,2.228039e-04,0.000651,6.507064e-04,g>1_diff
148,LTC-BTC-BRL,LTCBTC,0.001584,0.001585,LTCBRL,339.70,340.10,BTCBRL,2.145410e+05,2.145710e+05,3.400950e+02,0.999985,1.000391,1.459865e-05,0.000391,3.913571e-04,g>1_diff
141,LTC-BTC-TUSD,LTCBTC,0.001584,0.001585,LTCTUSD,68.87,68.96,BTCTUSD,4.349532e+04,4.349533e+04,6.894010e+01,0.999711,1.000386,2.886014e-04,0.000386,3.860444e-04,g>1_diff
